In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.stats.multicomp as multi
from dataFilter import Replacing

df = pd.read_excel("C:/Users/Админ/Documents/GitHub/DANO_NN_2024/Data/hakaton_nn_1month.xlsx")

df["education_level_cd"] = df["education_level_cd"].replace("SCH",0)
df["education_level_cd"] = df["education_level_cd"].replace(";",0)
df["education_level_cd"] = df["education_level_cd"].replace("UGR",1)
df["education_level_cd"] = df["education_level_cd"].replace("GRD",2)
df["education_level_cd"] = df["education_level_cd"].replace("PGR",3)
df["education_level_cd"] = df["education_level_cd"].replace("ACD",4)
df["education_level_cd"] = df["education_level_cd"].replace("MGR",5)
df["education_level_cd"] = df["education_level_cd"].replace("AGR",6)
df["education_level_cd"] = df["education_level_cd"].replace("SPC",7)
df["education_level_cd"] = df["education_level_cd"].fillna(1)
df["education_level_cd"] = df["education_level_cd"].astype(int)

df["marital_status_cd"] = df["marital_status_cd"].replace(";",0)
df["marital_status_cd"] = df["marital_status_cd"].replace("MAR",2)
df["marital_status_cd"] = df["marital_status_cd"].replace("DIV",-2)
df["marital_status_cd"] = df["marital_status_cd"].replace("UNM",-1)
df["marital_status_cd"] = df["marital_status_cd"].replace("WID",-1)
df["marital_status_cd"] = df["marital_status_cd"].replace("CIV",1)
df["marital_status_cd"] = df["marital_status_cd"].replace("DLW",0)
df["marital_status_cd"] = df["marital_status_cd"].fillna(0)
df["marital_status_cd"] = df["marital_status_cd"].astype(int)

df["children_cnt"] = df["children_cnt"].fillna(0)
df["children_cnt"] = df["children_cnt"].astype(int)
df["person_monthly_income_amt"] = df["person_monthly_income_amt"].fillna(100)
df["person_monthly_income_amt"] = df["person_monthly_income_amt"].astype(int)

df["day_of_week"] = df["day_of_week"].replace("Понедельник",0)
df["day_of_week"] = df["day_of_week"].replace("Вторник",1)
df["day_of_week"] = df["day_of_week"].replace("Среда",2)
df["day_of_week"] = df["day_of_week"].replace("Четверг",3)
df["day_of_week"] = df["day_of_week"].replace("Пятница",4)
df["day_of_week"] = df["day_of_week"].replace("Суббота",5)
df["day_of_week"] = df["day_of_week"].replace("Воскресенье",6)
df["day_of_week"] = df["day_of_week"].fillna(2)

df["age"] = df["age"].fillna(25)

df["hours"] = list(map(int,[str(i).split(':')[0] for i in df["offencetime"]]))

df["public_holiday"] = df["public_holiday"].replace('t',1)
df["public_holiday"] = df["public_holiday"].replace('f',0)

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


In [89]:
df["offenceprob"] = np.zeros(len(df["children_cnt"]))
for i in range(len(df["children_cnt"])):
    df["offenceprob"][i] = len(df["age"][(df["children_cnt"] == df["children_cnt"][i]) & (df["person_monthly_income_amt"] == df["person_monthly_income_amt"][i])
                                         & (df["public_holiday"] == df["public_holiday"][i]) & (df["offencetime"]==df["offencetime"][i])])

print(df["offenceprob"])

sns.histplot(df["offenceprob"])

C:\Users\Админ\AppData\Local\Temp\ipykernel_7484\870361549.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["offenceprob"][i] = len(df["age"][(df["children_cnt"] == df["children_cnt"][i]) & (df["person_monthly_income_amt"] == df["perso

KeyboardInterrupt: 

In [88]:
import statsmodels.api as sm
import datetime as dt
from statsmodels.regression.linear_model import OLS

df["hours"] = list(map(int,[str(i).split(':')[0] for i in df["offencetime"]]))
df["hours"] = df["hours"]
df["hours"] = df["hours"].fillna(17)
df["public_holiday"] = df["public_holiday"]
df["public_holiday"] = df["public_holiday"].fillna(0)
df["children_cnt"] = df["children_cnt"]
df["children_cnt"] = df["children_cnt"].fillna(1)
df["person_monthly_income_amt"] = df["person_monthly_income_amt"]
df["person_monthly_income_amt"] = df["person_monthly_income_amt"].fillna(175)

X = np.array(df[["public_holiday","children_cnt","person_monthly_income_amt","hours"]])
Y = np.array(df["offenceprob"])
X = sm.add_constant(X,prepend=False)
model = OLS(Y,X)    
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     5560.
Date:                Sun, 22 Sep 2024   Prob (F-statistic):               0.00
Time:                        10:45:51   Log-Likelihood:                -183.75
No. Observations:               97307   AIC:                             377.5
Df Residuals:                   97302   BIC:                             424.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3084      0.059      5.219      0.0